In [33]:
import scipy 
import numpy as np
from pyDOE import *
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import tree
import sklearn.linear_model
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import sklearn.metrics
import time
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier




input_size = 24

def sigmoid(z):
    return 1.0/(1.0 + np.exp(-z))

def Loss(x,y):
    return (x - y)**2




    
#sigmoid!!
def forecast(w, x, c):

    return activation(w[5 * (2 + input_size)] +\
                      (activation( w[5 * input_size: 5 * (input_size + 1)] +\
                                  np.matrix(x) * w[:5 * input_size].reshape(input_size, 5),\
                                  c)).dot(w[5 * (1 + input_size):5 * (2 + input_size)]), c)    
    
def act1(z):
    return (1.0 + np.tanh(z))/2.0
    
def act2(z):
    return ((2.0 * np.arctan(z)/np.pi) + 1.0)/2.0
    
def activation(z, c):    
    if (c == 'arctan'):
        return act2(z)
    if (c == 'tanh'):
        return act1(z)
    return sigmoid(z)

#Generating the train set and scaling it
#For function  (1 + sin(x))/2 

#N = number of points
#M = number of generated expectations for each x
#K = number of instances W for each expectation

def learn_network(c, M = 10, K = 20):
    
    #we generate the training set
    #Train_x = np.linspace(-np.pi, np.pi, N)
    #Train_x_scaled = np.linspace(-1, 1, N)
    #Train_y = ( 1.0 + np.sin(Train_x_scaled)) / 2.0
    
    
    
    """
    #X
    f1 = open('madelon_train.data.txt', 'r')
    c = f1.readlines()
    #!!!!!!!!!!len(c)
    L = [c[i].split(" ")[:input_size] for i in range(1000)]
    L = [[float(L[i][j]) for j in range(len(L[i]))] for i in range(len(L))]
    Train_x = np.matrix(L)
    f1.close()
    
    #Y
    #function to approximate
    f2 = open('madelon_train.labels.txt', 'r')   
    
    c = f2.readlines()
    #len(c)
    L = [float(c[i].split("\n")[0]) for i in range(1000)]
    L = [[1.0] if L[i] == 1.0 else [0.0] for i in range(len(L))]
    Train_y = np.matrix(L)
    f2.close()
    
    """
    #*********************************************
    #*************OLD DATASET*********************************
    """
    f = open("pima-indians-diabetes.data.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(200)]
    l = [l[i][0].split(',') for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Train_x = np.matrix([l[i][:8] for i in range(len(l))])
    Train_y = np.matrix([[l[i][8]] for i in range(len(l))])
    #Train_y = ( np.multiply(Train_x,Train_x) * np.matrix([[1],[1]])) / 2.0
    #print([y[0,0] == 0.0 for y in Train_y])
    """
    #*****************Second DATASET******************************
    #***********************************************
    """
    f = open("datatraining.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(1,len(l))]
    l = [l[i][0].split(',')[2:8] for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    Train_x = np.matrix([l[i][:5] for i in range(len(l))])
    Train_y = np.matrix([[l[i][5]] for i in range(len(l))])
    """
    #*****************Third DATASET******************************
    #***********************************************
    """
    f = open("ionosphere.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(170)]
    l = [l[i][0].split(',') for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][34] == 'g'):
            l[i][34] = 1.0
        else:
            l[i][34] = 0.0
        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Train_x = np.matrix([l[i][:34] for i in range(len(l))])
    Train_y = np.matrix([[l[i][34]] for i in range(len(l))])
    #Train_y = ( np.multiply(Train_x,Train_x) * np.matrix([[1],[1]])) / 2.0
    
    if (l[i][24] == '1'):
        l[i][24] = 1.0
    else:
        l[i][24] = 0.0
    """
    #*****************Fourth DATASET******************************
    #***********************************************
    f = open("herman.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(500)]
    l = [l[i][0].split(',')[:25] for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][24] == '1'):
            l[i][24] = 1.0
        else:
            l[i][24] = 0.0

        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Train_x = np.matrix([l[i][:24] for i in range(len(l))])
    Train_y = np.matrix([[l[i][24]] for i in range(len(l))])

    
    
    
    
    
    #we scale dataset or not
    scaler = preprocessing.StandardScaler().fit(Train_x);
    
    #Train_x_scaled = np.array(Train_x.copy())
    Train_x_scaled = scaler.transform(Train_x);
    
    
    
    #Visualizing dataset
    
    trues = Train_x_scaled[[y[0,0] == 1.0 for y in Train_y]]
    falses = Train_x_scaled[[y[0,0] == 0.0 for y in Train_y]]
    
    comp_x = 3
    comp_y = 4
    x_true = [trues[i,comp_x] for i in range(len(trues))]
    y_true = [trues[i,comp_y] for i in range(len(trues))]
    x_false = [falses[i,comp_x]for i in range(len(falses))]
    y_false = [falses[i,comp_y] for i in range(len(falses))]
    
    
    #plt.xlabel('Train LL(random partition) ')
    line1 = plt.plot(x_true, y_true, 's',  color = 'r', label = 'True') 
    line2 = plt.plot(x_false, y_false, '*',  color = 'b', label = 'False')
    #plt.xscale('log')
    plt.legend(loc='down left')
    #plt.savefig('rand_train.png')

    plt.plot()
    plt.show()
    
    
    #Dictionary of type {point:best_expectation_of_parameters}
    Pairs = {}
    
    #Disctionary of type {}
    Losses = {}
    
    #Variance parameter
    sigma = 1.0
    Diag = [[sigma if r==s else 0 for r in range(5 * (2 + input_size) + 1)] for s in range(5 * (2 + input_size) + 1)]
    #for each x we find best fitting vector of expectation
    
    
    
    for i in range(len(Train_x_scaled)):
        
        
        if (i % 10 == 0) :
            print( "step " + str(i) + " of " + str(len(Train_x_scaled)) + "...")
        
        #we generate a random set of parameters and scale them to [0,1]^10
        
        v_set = 2.0 * lhs(5 * (2 + input_size) + 1, M, criterion='maximin') - 1.0
        #maps average error to expectation_of_parameters(vector)
        error_to_pair = {}
         
        for j in range(M):#For every vector of parameters(expectation)
        
            #SECTION 1
            
            #SetW = [0 for q in range(K)]
            
            SetW = np.random.multivariate_normal((v_set[j,:]), Diag, K)
                       
            #losses = [0 for q in range(K)]
            
            
            #for q in range(K):#we calculate loss on each instance of W
                #sigmoid!!!   
            losses =  map(lambda A: Loss(activation(A[5 * (2 + input_size)] +\
            activation(A[5 * (input_size) : 5 * (1 + input_size)] +\
            np.matrix(Train_x_scaled[i]) * A[ : 5 *  input_size].reshape(input_size,5), c).dot(A[5 * (1\
            + input_size):5*(2 + input_size)]), c), Train_y[i,0]), SetW )
                
                
            #we calculate average error for this instance of expectation vector
            Average_error = (np.array(losses)).mean()
            
            #error -> parameter of exspectation realising it
            error_to_pair[Average_error] = v_set[j,:]
            
            
        
        #we find the expectation that gives the minimum expected error and attach it the x 
        min_error = min(error_to_pair.keys())
        
        Pairs[i] = error_to_pair[min_error]
        Losses[i] = min_error   
    
    s = Pairs[0].copy()
        
   
    #Choice of classifier
    #clf = tree.DecisionTreeRegressor(max_depth = 8);
    clf = sklearn.linear_model.LinearRegression(fit_intercept=False, normalize = False, copy_X=True)


    clf = clf.fit([[Train_x_scaled[i][j] for j in range(input_size)]\
                   for i in range(Train_x_scaled.shape[0])], [Pairs[i] for i in range(Train_x_scaled.shape[0])]);
    

    
    #Printing parameters
    numer_of_parameter = 47
    """
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    c1 = 6
    c2 = 7
    p1 = ax.scatter([x[c1] for x in trues], [x[c2] for x in trues],\
                    [Pairs[i][numer_of_parameter] for i in range(len(Train_x_scaled)) if Train_y[i] == 1.0 ], s = 100, c = "r")
    
    p2 = ax.scatter([x[c1] for x in falses], [x[c2] for x in falses],\
                    [Pairs[i][numer_of_parameter] for i in range(len(Train_x_scaled)) if Train_y[i] == 0.0 ], s = 100, c = "g")
    
    pd = clf.predict([[x[i] for i in range(x.shape[0])] for x in Train_x_scaled])
    
    p3 = ax.plot_trisurf([Train_x_scaled[i,c1] for i in range(Train_x_scaled.shape[0])],\
                         [Train_x_scaled[i,c2] for i in range(Train_x_scaled.shape[0])],\
                         [pd[i][numer_of_parameter] for i in range(len(Train_x_scaled))])
    
    ax.set_xlabel("x[ "+ str(c1)+ " ]")
    ax.set_ylabel("x[ " + str(c2)+ " ]")
    ax.set_zlabel("Parameter")
    ax.set_title("Input" + str(c1) + str(c2)+'Parameter' + str(numer_of_parameter) + '(x)')
    
    plt.show()
    plt.savefig("Input" + str(c1) + str(c2)+'Parameter' + str(numer_of_parameter) + ".jpg", dpi=1000)
    #plt.legend((p1,p2),('Best Parameters', 'Fitted parameters'),numpoints=1, loc='Higer left', ncol=2, fontsize=8,  bbox_to_anchor=(0.8, 0.3))
    """

    
    
    
    
    
    
    test(scaler, Train_x_scaled, Train_y, clf, c, input_size)    
    #return (scaler, Train_x_scaled, Train_y, Pairs, clf)
        

def test(scaler, Train_x_scaled, Train_y, clf, c, input_size):
    
    #Test_x_1 = np.linspace(-1.0,1.0 ,13)
    #Test_x_2 = np.linspace(-1.0, 1.0, 13)
    #Test_x = np.matrix([[x,y] for x in Test_x_1 for y in Test_x_2])
    #Test_y = ( 1.0 + np.multiply(Test_x,Test_x) * np.matrix([[1],[1]])) / 2.0

    #Test_x_scaled = scaler.transform(Test_x);
    #Test_x = np.linspace(-np.pi, np.pi, 23)
    
    #Test_x_scaled = scaler.transform(Test_x)

    
    #*****************OLD****************************************
    #*********************************************************
    #********************************************************
    """
    f = open("pima-indians-diabetes.data.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(200,len(l))]
    l = [l[i][0].split(',') for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]    
    
    
    Test_x = np.matrix([l[i][:8] for i in range(len(l))])
    Test_y = np.matrix([[l[i][8]] for i in range(len(l))])
    """
    
    
    #*****************NEW DATASET******************************
    #***********************************************
    """
    f = open("datatest.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(1,len(l))]
    l = [l[i][0].split(',')[2:8] for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    Test_x = np.matrix([l[i][:5] for i in range(len(l))])
    Test_y = np.matrix([[l[i][5]] for i in range(len(l))])
    
    """
    
    #***************Third dataset******************
    #*********************************************
    """
    f = open("ionosphere.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(170,len(l))]
    l = [l[i][0].split(',') for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][34] == 'g'):
            l[i][34] = 1.0
        else:
            l[i][34] = 0.0
        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    """
       #*****************Fourth DATASET******************************
    #***********************************************
    f = open("herman.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(500,1000)]
    l = [l[i][0].split(',')[:25] for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][24] == '1'):
            l[i][24] = 1.0
        else:
            l[i][24] = 0.0

        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Test_x = np.matrix([l[i][:24] for i in range(len(l))])
    Test_y = np.matrix([[l[i][24]] for i in range(len(l))])
    

    
    
    
    
    #Test_x_scaled = np.array(Test_x.copy())
    Test_x_scaled = scaler.transform(Test_x);
    
    
    pd = clf.predict([[x[i] for i in range(x.shape[0])] for x in Test_x_scaled])
    
    
    predictions = np.zeros(Test_x.shape[0])
    
    for i in range(Test_x.shape[0]):
        predictions[i] = forecast(pd[i], Test_x_scaled[i], c)
    """
    p1 = [Train_x_scaled[i,1] for i in range(len(Train_x_scaled))]
    p2 = [Train_x_scaled[i,3] for i in range(len(Train_x_scaled))]
    yp = [Train_y[i,0] for i in range(len(Train_y))]
    print(yp[1])
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    #q1 = ax.plot_trisurf([Train_x_scaled[i,0] for i in range(Train_x_scaled.shape[0])],\
    [Train_x_scaled[i,1] for i in range(Train_x.shape[0])], [Pairs[i][numer_of_parameter] for i in range(len(Pairs.keys()))], s = 3, c = "g")
    q2 = ax.scatter(p1,p2,yp,cmap=cm.bone, alpha=0.05)
    plt.plot()    
    """
    print("LogLoss per unit is " + str(sklearn.metrics.log_loss(Test_y, predictions, normalize = True)));
    
    predictions = [1  if predictions[i] > 0.5 else 0 for i in range(len(predictions))]
    
    print("Accuracy is " + str(sklearn.metrics.accuracy_score(Test_y, predictions, normalize = True)));
    

    #print("Predictions " + c + "!********************")
    #for i in range(Test_x_scaled.shape[0]):
    #    print(predictions[i])
    
    

    #fig = plt.figure()
    #ax = fig.add_subplot(111, projection='3d')
    
    #ax.plot_trisurf([Train_x_scaled[i][0] for i in range(Train_x_scaled.shape[0])],\
    #[Train_x_scaled[i][1] for i in range(Train_x_scaled.shape[0])],[x[0,0] for x in Train_y],cmap=cm.bone, alpha=0.4 )
    #ax.scatter([Test_x_scaled[i][0] for i in range(Test_x_scaled.shape[0])],[Test_x_scaled[i][1] for i in range(Test_x_scaled.shape[0])],predictions, s = 3, c = "g")
    #fig.savefig(c + "3d.png")
    #plt.plot(Test_x_scaled, predictions,'o', Train_x_scaled, Train_y)
    #plt.title("Activation function: " + c)
    #plt.savefig(c + "3d.png")

def read_data():
    
    f = open('madelon_train.data.txt', 'r')
    
    c = f.readlines()
    L = [c[i].split(" ")[:500] for i in range(len(c))]
    L = [[float(L[i][j]) for j in range(len(L[i]))] for i in range(len(L))]
    
    
    #for y
    L = [float(c[i].split("\n")[0]) for i in range(len(c))]




def network(c = "cls"):
    

    
    #Train_y = ( np.multiply(Train_x,Train_x) * np.matrix([[1],[1]])) / 2.0
    #Train_x_scaled = np.array(Train_x.copy())
    
    #First
    """
    f = open("pima-indians-diabetes.data.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(100)]
    l = [l[i][0].split(',') for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    Train_x = np.matrix([l[i][:8] for i in range(len(l))])
    Train_y = np.matrix([[l[i][8]] for i in range(len(l))])
    f.close()
    """
    #Second
    """
    f = open("datatest.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(1,len(l))]
    l = [l[i][0].split(',')[2:8] for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    Train_x = np.matrix([l[i][:5] for i in range(len(l))])
    Train_y = np.matrix([[l[i][5]] for i in range(len(l))])
    f.close()
    """
    #Third
    """
    f = open("ionosphere.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(170)]
    l = [l[i][0].split(',') for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][34] == 'g'):
            l[i][34] = 1.0
        else:
            l[i][34] = 0.0
        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    f.close()
    """
    #*****************Fourth DATASET******************************
    #***********************************************
    f = open("herman.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(0,500)]
    l = [l[i][0].split(',')[:25] for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][24] == '1'):
            l[i][24] = 1.0
        else:
            l[i][24] = 0.0

        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Train_x = np.matrix([l[i][:24] for i in range(len(l))])
    Train_y = np.matrix([[l[i][24]] for i in range(len(l))])
    
    f.close()


    
    scaler = preprocessing.StandardScaler().fit(Train_x)
    
    
    #we scale dataset or not
    #Test_x_scaled = np.array(Test_x.copy())
    Train_x_scaled = scaler.transform(Train_x)
    
    if (c == "reg"):
        clf = MLPRegressor(activation='tanh', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5))
    else:
        clf = MLPClassifier(activation='tanh', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5))
    
    clf.fit(Train_x_scaled, Train_y)
    
    #OLD
    """
    f = open("pima-indians-diabetes.data.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(100, len(l))]
    l = [l[i][0].split(',') for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Test_x = np.matrix([l[i][:8] for i in range(len(l))])
    Test_y = np.matrix([[l[i][8]] for i in range(len(l))])
    f.close() 
    """
    #NEW
    """
    f = open("datatest.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(1,len(l))]
    l = [l[i][0].split(',')[2:8] for i in range(len(l))]
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    Test_x = np.matrix([l[i][:5] for i in range(len(l))])
    Test_y = np.matrix([[l[i][5]] for i in range(len(l))])
    f.close() 
    """
    #Third
    """
    f = open("ionosphere.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(170,len(l))]
    l = [l[i][0].split(',') for i in range(len(l))]
    
    for i in range(len(l)):
        if (l[i][34] == 'g'):
            l[i][34] = 1.0
        else:
            l[i][34] = 0.0

        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    f.close()
    """
    f = open("herman.txt",'r')
    l = f.readlines()
    l = [l[i].split("\n") for i in range(500,1000)]
    l = [l[i][0].split(',')[:25] for i in range(len(l))]
    for i in range(len(l)):
        if (l[i][24] == '1'):
            l[i][24] = 1.0
        else:
            l[i][24] = 0.0

        
    l = [[float(l[i][j]) for j in range(len(l[i]))] for i in range(len(l))]
    
    Test_x = np.matrix([l[i][:24] for i in range(len(l))])
    Test_y = np.matrix([[l[i][24]] for i in range(len(l))])
    f.close() 
    
    
    
    #We scale dataset or not
    #Test_x_scaled = np.array(Test_x.copy())
    Test_x_scaled = scaler.transform(Test_x)
      
    
    
    
    prediction = clf.predict(Test_x_scaled)
    
    #print("Precision is " + str(sklearn.metrics.accuracy_score(Test_y, prediction, normalize = True)))
    if (c == "reg"):
        print("Logloss is " + str(sklearn.metrics.log_loss(Test_y, prediction, normalize = True)))
    else:
        print("Accuracy is  " + str(sklearn.metrics.accuracy_score(Test_y, prediction, normalize = True)))
        
    
    #print(clf.coefs_[0])
    
    
    





In [37]:
learn_network("arctan")

step 0 of 500...
step 10 of 500...
step 20 of 500...
step 30 of 500...
step 40 of 500...
step 50 of 500...
step 60 of 500...
step 70 of 500...
step 80 of 500...
step 90 of 500...
step 100 of 500...
step 110 of 500...
step 120 of 500...
step 130 of 500...
step 140 of 500...
step 150 of 500...
step 160 of 500...
step 170 of 500...
step 180 of 500...
step 190 of 500...
step 200 of 500...
step 210 of 500...
step 220 of 500...
step 230 of 500...
step 240 of 500...
step 250 of 500...
step 260 of 500...
step 270 of 500...
step 280 of 500...
step 290 of 500...
step 300 of 500...
step 310 of 500...
step 320 of 500...
step 330 of 500...
step 340 of 500...
step 350 of 500...
step 360 of 500...
step 370 of 500...
step 380 of 500...
step 390 of 500...
step 400 of 500...
step 410 of 500...
step 420 of 500...
step 430 of 500...
step 440 of 500...
step 450 of 500...
step 460 of 500...
step 470 of 500...
step 480 of 500...
step 490 of 500...
LogLoss per unit is 0.605450101576
Accuracy is 0.7


In [18]:
s = np.array([True, False])
s/True

array([1, 0], dtype=int8)

In [42]:
network(c = "cls")

Accuracy is  0.7
